In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

# OBSERVABLE MODEL

In [ ]:
# Define the parameters
μ = 1.05
L1 = 1
L2 = 1
R1 = 15
c1 = 6
R2 =30
c2 = 5

### Calculating Total Revenue: 

In [ ]:
def calculate_revenue(P,μ, L1, L2, R1, c1, R2, c2):
    
    #finding n*
    n_star = math.floor(μ * ((R1-P) / c1))
    if n_star < 0:
        n_star = 0
    
   #finding m*
    m_star = math.floor((μ - L1) * ((R2-P) / c2))
    if m_star < 0:
        m_star = 0

    #creating states for type-2 markov chain 
    pairs = [(a, b) for a in range(int(n_star)+1) for b in range(int(m_star)+1) if a + b <= int(n_star + m_star)]
    pairs = [(a, b) for a, b in pairs if a <= n_star]
    
    states = []
    for a, b in pairs:
        if a == 0 and b == 0:
            states.append((a, b, 0))
            states.append((a, b, 1))
        else:
            states.append((a, b, 1))
    
    #creating transition matrix for type-2 markov chain
    total_states = len(states)
    transition_matrix = np.zeros((total_states, total_states))

    for idx, (i, j, k) in enumerate(states):
        current_state = idx

        if k==0:
            next_state = states.index((0, 0, 1)) 
            if P > R1 and P <= R2:
                transition_matrix[current_state, next_state] = L2
            elif P > R2 and P <= R1:
                transition_matrix[current_state, next_state] = L1
            else:
                transition_matrix[current_state, next_state] = L1+L2      
            
        if k!=0 and i==0 and j==0:
            next_state = states.index((0,0,0))
            transition_matrix[current_state, next_state] = μ
            
            if k+i<=n_star and P <= R1:
                next_state = states.index((1,0,1))
                transition_matrix[current_state, next_state] = L1
            
            if k+i+j<=m_star and P <= R2:
                next_state = states.index((0,1,1))
                transition_matrix[current_state, next_state] = L2   
                         
        if j==0 and i!=0:
            next_state = states.index((i-1, j, 1)) if (i-1, j, 1) in states else None
            if next_state is not None:
                transition_matrix[current_state, next_state] = μ
            
            if k+i<=n_star and P <= R1 :
                next_state = states.index((i+1, j, 1)) if (i+1, j, 1) in states else None
                if next_state is not None:
                    transition_matrix[current_state, next_state] = L1
            
            if k+i+j<=m_star and P <= R2:
                next_state = states.index((i, j+1, 1)) if (i, j+1, 1) in states else None
                if next_state is not None:
                    transition_matrix[current_state, next_state] = L2

        if j!=0 and i==0:
            next_state = states.index((i, j-1, 1)) if (i, j-1, 1) in states else None
            if next_state is not None:
                transition_matrix[current_state, next_state] = μ
                
            if k+i<=n_star and P <= R1:
                next_state = states.index((i+1, j, 1)) if (i+1, j, 1) in states else None
                if next_state is not None:
                    transition_matrix[current_state, next_state] = L1
            
            if k+i+j<=m_star and P <= R2:
                next_state = states.index((i, j+1, 1)) if (i, j+1, 1) in states else None
                if next_state is not None:
                    transition_matrix[current_state, next_state] = L2
        
        if j!=0 and i!=0:
            next_state = states.index((i, j-1, 1)) if (i, j-1, 1) in states else None
            if next_state is not None:
                transition_matrix[current_state, next_state] = μ
            
            next_state = states.index((i-1, j, 1)) if (i-1, j, 1) in states else None
            if next_state is not None:
                transition_matrix[current_state, next_state] = μ
    
            if k+i<=n_star and P <= R1:
                next_state = states.index((i+1, j, 1)) if (i+1, j, 1) in states else None
                if next_state is not None:
                    transition_matrix[current_state, next_state] = L1
            
            if k+i+j<=m_star and P <= R2:
                next_state = states.index((i, j+1, 1)) if (i, j+1, 1) in states else None
                if next_state is not None:
                    transition_matrix[current_state, next_state] = L2
            
    #Creating Q Matrix and solving for steady-state probs. of type-2 markov chain
    #https://vknight.org/blog/posts/continuous-time-markov-chains/
    n = transition_matrix.shape[0]
    adjusted_matrix = transition_matrix.copy()
    for i in range(n):
        adjusted_matrix[i, i] = -np.sum(transition_matrix[i, :])    
    Q=adjusted_matrix
    dimension = Q.shape[0]
    M = np.vstack((Q.transpose()[:-1], np.ones(dimension)))
    b = np.vstack((np.zeros((dimension - 1, 1)), [1]))
    state=np.linalg.solve(M, b).transpose()[0]
    type2_probs=[]
    for i in state:
        type2_probs.append(i)
    result_dict = dict(zip(states, type2_probs))

    #Type-1 steady-state probs. using the formulas from type-1 Markov chain
    type1_probs = {}
    π0 = (μ - L1) / (μ + L2 - (L1 + L2) * ((L1 / μ) ** (n_star+1)))
    type1_probs["π0"] = π0
    for n in range(1, n_star + 2):
        π_n = π0 * ((L1 / μ) ** (n - 1)) * ((L1 + L2) / μ)
        type1_probs[f"π{n}"] = π_n
    result = type1_probs

    #Type-1 Joining rate and equilibrium lambda
    eq_L1 = L1*(1-result[f"π{n_star+1}"])
    if P>R1:
        eq_L1=0
    
    #Type-2 joining rate equilibrium lambda
    type2_joining_rate = 0
    for state in states:
        if sum(state) <= m_star:
            type2_joining_rate += result_dict[state]
    if P>R2:
        type2_joining_rate = 0
                  
    eq_L2 = L2*type2_joining_rate
       
    #Total revenue
    Revenue=(eq_L1+eq_L2)*P
    
    
    return Revenue, eq_L1, eq_L2

### Revenue Maximization:

In [ ]:
#Revenue Maximization
P_values = np.linspace(0, max(R1, R2), 10*(max(R1, R2))+1)
max_revenue = float('-inf')
optimal_P = None
    
for P in P_values:
    revenue = calculate_revenue(P, μ, L1, L2, R1, c1, R2, c2)[0]
    if revenue > max_revenue:
        max_revenue = revenue
        optimal_P = P

print("Optimal Price:", optimal_P)
print("Maximum Revenue:", max_revenue)
print("Type-1 Entering Rate:",calculate_revenue(optimal_P, μ, L1, L2, R1, c1, R2, c2)[1])
print("Type-2 Entering Rate:",calculate_revenue(optimal_P, μ, L1, L2, R1, c1, R2, c2)[2])

### Calculating Total Utility: 

In [ ]:
def calculate_utility(P,μ, L1, L2, R1, c1, R2, c2):
    
    
    n_star = math.floor(μ * ((R1-P) / c1))
    if n_star < 0:
        n_star = 0
    
    
    m_star = math.floor((μ - L1) * ((R2-P) / c2))
    if m_star < 0:
        m_star = 0


    pairs = [(a, b) for a in range(int(n_star)+1) for b in range(int(m_star)+1) if a + b <= int(n_star + m_star)]
    pairs = [(a, b) for a, b in pairs if a <= n_star]
    
    states = []
    for a, b in pairs:
        if a == 0 and b == 0:
            states.append((a, b, 0))
            states.append((a, b, 1))
        else:
            states.append((a, b, 1))
            
  
    total_states = len(states)
    transition_matrix = np.zeros((total_states, total_states))

    for idx, (i, j, k) in enumerate(states):
        current_state = idx

      
        if k==0:
            next_state = states.index((0, 0, 1)) 
            if P > R1 and P <= R2:
                transition_matrix[current_state, next_state] = L2
            elif P > R2 and P <= R1:
                transition_matrix[current_state, next_state] = L1
            else:
                transition_matrix[current_state, next_state] = L1+L2
            
                
            
        if k!=0 and i==0 and j==0:
            next_state = states.index((0,0,0))
            transition_matrix[current_state, next_state] = μ
            
            if k+i<=n_star and P <= R1:
                next_state = states.index((1,0,1))
                transition_matrix[current_state, next_state] = L1
            
            if k+i+j<=m_star and P <= R2:
                next_state = states.index((0,1,1))
                transition_matrix[current_state, next_state] = L2
            
                         
        if j==0 and i!=0:
            next_state = states.index((i-1, j, 1)) if (i-1, j, 1) in states else None
            if next_state is not None:
                transition_matrix[current_state, next_state] = μ
            
            if k+i<=n_star and P <= R1 :
                next_state = states.index((i+1, j, 1)) if (i+1, j, 1) in states else None
                if next_state is not None:
                    transition_matrix[current_state, next_state] = L1
            
            if k+i+j<=m_star and P <= R2:
                next_state = states.index((i, j+1, 1)) if (i, j+1, 1) in states else None
                if next_state is not None:
                    transition_matrix[current_state, next_state] = L2

        if j!=0 and i==0:
            next_state = states.index((i, j-1, 1)) if (i, j-1, 1) in states else None
            if next_state is not None:
                transition_matrix[current_state, next_state] = μ
                
            if k+i<=n_star and P <= R1:
                next_state = states.index((i+1, j, 1)) if (i+1, j, 1) in states else None
                if next_state is not None:
                    transition_matrix[current_state, next_state] = L1
            
            if k+i+j<=m_star and P <= R2:
                next_state = states.index((i, j+1, 1)) if (i, j+1, 1) in states else None
                if next_state is not None:
                    transition_matrix[current_state, next_state] = L2
        
        if j!=0 and i!=0:
            next_state = states.index((i, j-1, 1)) if (i, j-1, 1) in states else None
            if next_state is not None:
                transition_matrix[current_state, next_state] = μ
            
            next_state = states.index((i-1, j, 1)) if (i-1, j, 1) in states else None
            if next_state is not None:
                transition_matrix[current_state, next_state] = μ
    
            if k+i<=n_star and P <= R1:
                next_state = states.index((i+1, j, 1)) if (i+1, j, 1) in states else None
                if next_state is not None:
                    transition_matrix[current_state, next_state] = L1
            
            if k+i+j<=m_star and P <= R2:
                next_state = states.index((i, j+1, 1)) if (i, j+1, 1) in states else None
                if next_state is not None:
                    transition_matrix[current_state, next_state] = L2
            
    #https://vknight.org/blog/posts/continuous-time-markov-chains/
    n = transition_matrix.shape[0]
    adjusted_matrix = transition_matrix.copy()
    for i in range(n):
        adjusted_matrix[i, i] = -np.sum(transition_matrix[i, :])   
    Q=adjusted_matrix
    dimension = Q.shape[0]
    M = np.vstack((Q.transpose()[:-1], np.ones(dimension)))
    b = np.vstack((np.zeros((dimension - 1, 1)), [1]))
    state=np.linalg.solve(M, b).transpose()[0]
    type2_probs=[]
    for i in state:
        type2_probs.append(i)
    result_dict = dict(zip(states, type2_probs))
    

    type1_probs = {}
    π0 = (μ - L1) / (μ + L2 - (L1 + L2) * ((L1 / μ) ** (n_star+1)))
    type1_probs["π0"] = π0
    for n in range(1, n_star + 2):
        π_n = π0 * ((L1 / μ) ** (n - 1)) * ((L1 + L2) / μ)
        type1_probs[f"π{n}"] = π_n
    result = type1_probs

    eq_L1 = L1*(1-result[f"π{n_star+1}"])
    if P>R1:
        eq_L1=0
        
    type2_joining_rate = 0
    for state in states:
        if sum(state) <= m_star:
            type2_joining_rate += result_dict[state]
    if P>R2:
        type2_joining_rate = 0
            
           
    eq_L2 = L2*type2_joining_rate
    
    
    U2=0
    for (n, m, k), value in result_dict.items():
        if n+m+k <= m_star:
            U2+=((R2-P)-c2*(n+m+k)/(μ-L1))*result_dict[(n, m, k)]
    
    
    U1=0
    for n in range(0, n_star+1):
        U1+=((R1-P) - c1*(n)/μ)*result[f"π{n}"]
    
    total_utility=U1*eq_L1+U2*eq_L2


    
    return total_utility, eq_L1, eq_L2

### Utility Maximization: 

In [ ]:
P_values = np.linspace(0, max(R1, R2), 10*(max(R1, R2))+1)
max_utility = float('-inf')
optimal_P = None 
for P in P_values:
    utility = calculate_utility(P, μ, L1, L2, R1, c1, R2, c2)[0]
    if utility > max_utility:
        max_utility = utility
        optimal_P = P


print("Optimal Price:", optimal_P)
print("Maximum Utility:", max_utility)
print("Type-1 Entering Rate:",calculate_utility(optimal_P, μ, L1, L2, R1, c1, R2, c2)[1])
print("Type-2 Entering Rate:",calculate_utility(optimal_P, μ, L1, L2, R1, c1, R2, c2)[2])

### Service Rate Optimization:

In [ ]:
# Service Rate Optimization 
c=1.5
μ_0 = μ
P_values = np.linspace(0, max(R1, R2), 10*(max(R1, R2))+1)
μ_values = [1.5, 2.0,  2.5, 3.0,  3.5, 4.0,  4.5, 5.0] 
max_revenue = float('-inf')
optimal_P = None
optimal_μ = None
    
for P in P_values:
    for μ in μ_values:
        revenue = calculate_revenue(P, μ, L1, L2, R1, c1, R2, c2)[0] - max(0,c*(μ-μ_0))
        if revenue > max_revenue:
            max_revenue = revenue
            optimal_P = P
            optimal_μ = μ
    
print("Optimal Price:", optimal_P)
print("Optimal rate:", optimal_μ)
print("Maximum Revenue:", max_revenue)
print("Type-1 Entering Rate:",calculate_revenue(optimal_P, optimal_μ, L1, L2, R1, c1, R2, c2)[1])
print("Type-2 Entering Rate:",calculate_revenue(optimal_P, optimal_μ, L1, L2, R1, c1, R2, c2)[2])

### Parameter Set Selection for the Experiments:

In [ ]:
μ_values = [1.1 , 1.5, 2, 3.1]
R_values = [5, 10, 15]
c1_values = [3, 4, 5]
c2_values = [1, 2, 3]
parameters = [(P,μ, L1, L2, R1, c1, R2, c2) for R1 in R_values for c1 in c1_values for R2 in R_values for c2 in c2_values for L1 in [1] for L2 in [1]  for μ in μ_values]